In [45]:
from pathlib import Path
import pandas as pd
import json


file_gd = Path(r"\\meetingroom\Meeting Room Share\Projects\I2G_Hub\I2G2023\station_lists_GD\gd_all.csv",  sep=r'\s+', engine='python')
file_sh1 =  Path(r"\\meetingroom\Meeting Room Share\Projects\I2G_Hub\I2G2023\station_lists_SH\region1.csv")
file_sh2 =  Path(r"\\meetingroom\Meeting Room Share\Projects\I2G_Hub\I2G2023\station_lists_SH\region2.csv")


In [46]:
# Read and clean the data
with open(file_gd, 'r') as f:
    data = f.readlines()
    data = [line.strip() for line in data]

# Process headers
headers = [h for h in data[0].split() if h]  # Assuming the first line is the header

# Process the rest of the data
cleaned_data = []
for line in data[1:]:  # Skip the header line
    # Split the line on spaces and filter out any empty strings
    parts = [part for part in line.split() if part]
    # Append to the list as a dictionary if the parts match the headers
    if len(parts) == len(headers):
        cleaned_data.append(dict(zip(headers, parts)))

# Create a DataFrame from the list of dictionaries
df_gd = pd.DataFrame(cleaned_data)

df_gd.head()


,Sta,Lon,Lat
0,GDXC,109.784212152513,21.0496557890283
1,GDWS,109.835629109638,20.5629357396005
2,GDSX,109.8542668142,21.2558946114921
3,GDOU,109.918819445236,21.3290164907557
4,GDQP,109.926260060921,21.627010262334


In [47]:
from Coord import ECEF
import numpy as np


df_sh1 = pd.read_csv(file_sh1, header= None)
df_sh1.drop(df_sh1.columns[[0,1,6]], inplace=True, axis=1)
df_sh1.head()

xyz_arr = df_sh1.iloc[:,1:].to_numpy()

ecef_arr = ECEF(xyz_arr)

lla_arr = ecef_arr.ecef2lla().position

a = np.array(df_sh1.iloc[:,0]).reshape(-1,1)
b = lla_arr[:,:-1]


comb_arr = np.hstack((a, b ))

df_sh1 = pd.DataFrame(comb_arr, columns = ['Sta', 'Lat', 'Lon'])
df_sh1['Lat'] = np.degrees(df_sh1['Lat'].astype(float))
df_sh1['Lon'] = np.degrees(df_sh1['Lon'].astype(float))



df_sh1.head()

,Sta,Lat,Lon
0,SHXJ,31.274665,121.271567
1,GXNN,22.796972,108.311875
2,GXBM,24.141795,107.253222
3,GXYY,23.923292,107.207632
4,GXTD,23.618149,107.129520


In [48]:
df_sh2 = pd.read_csv(file_sh2, header= None)
df_sh2.drop(df_sh2.columns[[0,1,6]], inplace=True, axis=1)
df_sh2.head()

xyz_arr = df_sh2.iloc[:,1:].to_numpy()

ecef_arr = ECEF(xyz_arr)

lla_arr = ecef_arr.ecef2lla().position

a = np.array(df_sh2.iloc[:,0]).reshape(-1,1)
b = lla_arr[:,:-1]


comb_arr = np.hstack((a, b ))

df_sh2 = pd.DataFrame(comb_arr, columns = ['Sta', 'Lat', 'Lon'])

# Convert "Lat" and "Lon" columns from radians to degrees
df_sh2['Lat'] = np.degrees(df_sh2['Lat'].astype(float))
df_sh2['Lon'] = np.degrees(df_sh2['Lon'].astype(float))

df_sh2.head()


,Sta,Lat,Lon
0,SHXJ,31.274665,121.271567
1,JSDY,32.013566,119.554561
2,JSJT,31.760683,119.634372
3,JSWJ,31.701124,119.878957
4,JSWU,31.586640,120.226687


In [49]:
full_df = pd.concat([df_gd, df_sh1, df_sh2], axis=0)

full_df.head()

,Sta,Lon,Lat
0,GDXC,109.784212152513,21.0496557890283
1,GDWS,109.835629109638,20.5629357396005
2,GDSX,109.8542668142,21.2558946114921
3,GDOU,109.918819445236,21.3290164907557
4,GDQP,109.926260060921,21.627010262334


In [50]:
# Reset the index to ensure uniqueness
full_df.reset_index(inplace=True)

full_df.set_index('Sta', inplace=True)

# remove duplicates
full_df = full_df[~full_df.index.duplicated(keep='first')]


# Convert 'Lon' and 'Lat' to float
full_df['Lon'] = pd.to_numeric(full_df['Lon'])
full_df['Lat'] = pd.to_numeric(full_df['Lat'])

print(full_df.head())

# Convert DataFrame to JSON
full_df.to_json(r"C:\Users\chcuk\Work\Projects\Tools\SkyMap\stn_list.json", orient="index")

      index         Lon        Lat
Sta                               
GDXC      0  109.784212  21.049656
GDWS      1  109.835629  20.562936
GDSX      2  109.854267  21.255895
GDOU      3  109.918819  21.329016
GDQP      4  109.926260  21.627010


In [51]:
# test read

stations = json.load(open(r"C:\Users\chcuk\Work\Projects\Tools\SkyMap\stn_list.json")).values()
stations

dict_values([{'index': 0, 'Lon': 109.7842121525, 'Lat': 21.049655789}, {'index': 1, 'Lon': 109.8356291096, 'Lat': 20.5629357396}, {'index': 2, 'Lon': 109.8542668142, 'Lat': 21.2558946115}, {'index': 3, 'Lon': 109.9188194452, 'Lat': 21.3290164908}, {'index': 4, 'Lon': 109.9262600609, 'Lat': 21.6270102623}, {'index': 5, 'Lon': 109.9349674305, 'Lat': 20.2557400169}, {'index': 6, 'Lon': 110.0718264683, 'Lat': 20.9181445526}, {'index': 7, 'Lon': 110.0718316672, 'Lat': 20.9181568151}, {'index': 8, 'Lon': 110.1274109347, 'Lat': 20.6765888347}, {'index': 9, 'Lon': 110.1631516076, 'Lat': 20.3417931772}, {'index': 10, 'Lon': 110.1980230699, 'Lat': 21.9465952712}, {'index': 11, 'Lon': 110.2327287375, 'Lat': 20.5055942568}, {'index': 12, 'Lon': 110.264080503, 'Lat': 21.5580283428}, {'index': 13, 'Lon': 110.2640832114, 'Lat': 21.5580340057}, {'index': 14, 'Lon': 110.3398885207, 'Lat': 20.7955641588}, {'index': 15, 'Lon': 110.3721180356, 'Lat': 21.2715264304}, {'index': 16, 'Lon': 110.3721265774, 'L

In [52]:
sh_df = pd.concat([df_sh1, df_sh2], axis=0)

# Reset the index to ensure uniqueness
sh_df.reset_index(inplace=True)

sh_df.set_index('Sta', inplace=True)
df_gd.set_index('Sta', inplace=True)

# remove duplicates
sh_df = sh_df[~sh_df.index.duplicated(keep='first')]

sh_df.to_json(r"C:\Users\chcuk\Work\Projects\Tools\SkyMap\sh_stn_list.json", orient="index")
df_gd.to_json(r"C:\Users\chcuk\Work\Projects\Tools\SkyMap\gd_stn_list.json", orient="index")